In [2]:
import gzip
import json
import glob
seekall = '/workspace/data/data/imagenet_fall11/fall11_whole.seekhelper.txt.gz'
def load_from_gz(filepath, tar_root_folder="", as_list_style=True):
    if as_list_style:
        info = []
    else:
        info = {}
    with gzip.open(filepath, 'rb') as f_in:
        for line in f_in:
            if as_list_style:
                info.append(tar_root_folder+line.decode('ascii'))
            else:
                l_sp = line.decode('ascii').split(':')
                info[l_sp[0]] = [int(l_sp[1]),int(l_sp[2])]
    return info

def load_from_seekhelper(dir0):
    if not dir0[-1] == '/':
        dir0+='/'
    return [dir0+p for p in load_from_gz(dir0+'.seekhelper.txt.gz')]
imgnet_alt_paths = load_from_seekhelper('/workspace/data/data/imagenet_alt/')
fashionpedia_paths = load_from_seekhelper('/workspace/data/data/fashionpedia/')
inat2019_paths = load_from_seekhelper('/workspace/data/inat2019/')
places365_paths = load_from_seekhelper('/workspace/data/places365/')
celeba_paths = load_from_seekhelper('/workspace/data/celeba/jpg256')

imgnet_root='/workspace/data/data/imagenet_fall11/'
imgnet_paths = load_from_seekhelper(imgnet_root)
coco_inp_dir = '/workspace/data/data/coco/images/train2017'
coco_paths = sorted(glob.glob(coco_inp_dir+'/*.jpg'))

In [3]:
import random
imgnet_paths1 = imgnet_paths[:]
random.shuffle(imgnet_paths1)
inat2019_paths1 = inat2019_paths[:]
random.shuffle(inat2019_paths1)
train_paths = [celeba_paths, coco_paths, imgnet_alt_paths, fashionpedia_paths, inat2019_paths1[:len(coco_paths)], places365_paths, imgnet_paths1[:len(coco_paths)]]
all_p = []
for t in train_paths:
    all_p += t
random.shuffle(all_p)

In [4]:
print([len(p) for p in train_paths])

[70000, 118287, 53542, 48823, 118287, 36500, 118287]


In [41]:
with open('training_paths.txt', 'wt') as f_out:
    for p in all_p:
        f_out.write(p.replace('\n','')+'\n')

In [42]:
!gzip training_paths.txt

In [45]:
len(all_p), len(all_p2)

(563726, 563726)

In [50]:
for i in range(len(all_p)):
    if all_p[i].replace('\n','') != all_p2[i].replace('\n',''):
        print(i,all_p[i], all_p2[i])
        break

In [5]:
#normalize sets and take smaller subsample
all_b = []
for t in train_paths:
    t0 = t[:]
    random.shuffle(t0)
    all_b += t0[:16384]

In [7]:
with open('ordered_paths_normalized.txt', 'wt') as f_out:
    for p in all_b:
        f_out.write(p.replace('\n','')+'\n')

In [8]:
!gzip ordered_paths_normalized.txt

In [43]:
!gunzip ordered_paths_normalized.txt.gz

In [ ]:
with open('ordered_paths_normalized.txt', 'r') as f_out:
    all_p2 = [p for p in f_out]